## Hopenhayn (1992, ECTA)


#### Environment

- Discrete time.
- Individual productivity $\varphi$ follows a 1st order markov process
- Entrants draw from some distribution $G(\varphi)$
- Firms are subject to an entry cost $c_e$ and a per period fixed cost $c_f$
- Firms' discount factor is given by $\beta \equiv 1/(1+r)$.
- Firms produce $q = f(\varphi, n)$. Assume: $q=\varphi f(n)$.
- Households will be represented by the inverse demand function $p=D(Q)$ where $D'(p)<0$. They supply labor inelastically.   
- Timing: 
    * incumbents decide to stay or exit / entrants decide to enter or not 
    * incumbents that stay pay $c_f$, entrants pay $c_e$
    * after paying the costs operating firms see their productivity

#### Static problem

The firm solves the following static problem, taking prices $p$ and $w$ as given: 

$\pi(\varphi; p, w) = \max_n \left\lbrace p\varphi f(n) - wn - wc_f\right\rbrace$.


#### Bellman Equation
The Bellman problem of an incumbent firm with productivity $\varphi$: 


$V_t(\varphi; p, w) = \pi(\varphi; p, w) + \beta\max\left\lbrace 0, \int V_{t+1}(\varphi'; p, w)dF(\varphi' | \varphi)\right\rbrace.$

#### Exit decision 
Firms will decide to exit if their productivity falls below their reservation productivity: $\varphi  < \varphi^*$. The cut-off solves (for interior cases):
 
$\int V_{t+1}(\varphi'; p, w)dF(\varphi' | \varphi^* ) = 0.$

#### Entry decision
Entrants don't know their true productivity beforehand. They pay to see it and begin producing next periods if they stay. Denote the mass of entrants by $M_t$. Free entry will give:

$V^e = \beta\int V_{t+1}(\varphi; p, w)dG(\varphi) - c_e \leq 0,$

with equality whenever $M_t > 0$.

#### Incumbent distribution 
Let the measure of incumbents be $\mu_t$. It evolves with the following law of motion:

$\mu_{t+1}([0,\phi^*))  = \int_{\varphi\geq\varphi^*}F(\varphi' | \varphi)\mu_t(d\varphi) + M_{t+1}G(\varphi).$

If we discretize the distribution, we can rewrite it in terms of the linear system: 

$\mu_{t+1} = P_t\mu_t + M_{t+1}g,$

where $\mu$ and $g$ are $Nx1$ vectors and $P_t$ is a $NxN$ matrix. 

#### Aggregate variables 
$Q^s(\mu, p, w) = \int q(\varphi; p, w) \mu(d\varphi),$

$N^d(\mu, p, w) = \int n(\varphi; p, w) \mu(d\varphi)$

#### Equilibrium
We will solve for the stationary competitive equilibrium. Normalize the wage to be the numeraire ($w = 1$) and define the policy decision to exit as $\chi(\varphi,p) \in \{0,1\}$, where 1 denotes the decision to exit. In recursive form, a stationary recursive competitive equilibrium  is a list of firms' policy function $(n(\varphi, p), \chi(\varphi, p))$, value functions $V, V^e$, price $p$, a measure of incumbent firms $\mu$, and a measure of entrants $M$, such that: 

- Given $p$, $n$ and $\chi$ solve the firm problem and the associated value function $V$.
- Free entry implies $V^e = 0$ if $M>0$ and $V^e<0$ if $M=0$.
- Goods market clear: $D(p) = Q^s(\mu, p) = \int q(\varphi, n; p) \mu(d\varphi)$
- The measure of incumbent plants is invariant and it solves: 
    
    \begin{align}
    \mu(d\varphi')  = \int_{\varphi\geq\varphi^*}F(\varphi' | \varphi)[1-\chi(\varphi,p)]\mu(d\varphi) + M G(\varphi)
    \end{align}

#### Functional forms

- Tecnology $y = \varphi n^\alpha$
- Markov chain is an AR(1): $\log \varphi = \overline{\varphi} + \rho \log \varphi + \sigma\varepsilon'$ where $\varepsilon \sim N(0,1)$
- Assume a simple demand of the form: $D = \overline{D}/p$

In [1]:
# P.S currently tested on 3.10.7 python version, but these are standard packages and should work in other versions as well.
import numpy as np
#import matplotlib.pyplot as plt  # if you need to plot 
#from numba import njit, jit, guvectorize # NUMBA speed up quite a lot, see the functions that have the decorator just above ====> not used in this code
from scipy.optimize import brentq  # root-finding routine
from scipy.stats import norm


In [92]:

def tauchen(N, rho, sigma, mu=0.0, m=3.0):
    s1 = mu/(1 - rho) - m * np.sqrt(sigma**2 / (1 - rho**2))
    sN = mu/(1 - rho) + m * np.sqrt(sigma**2 / (1 - rho**2))
    s = np.linspace(s1, sN, N) # grid values
    step = (s[N-1] - s[0]) / (N - 1)  # evenly spaced grid
    P = np.zeros((N, N))

    for i in range(np.ceil(N/2).astype(int)):
        P[i, 0] = norm.cdf((s[0] - mu - rho*s[i] + step/2) / sigma)
        P[i, N-1] = 1 - norm.cdf((s[N-1] - mu - rho*s[i] - step/2) / sigma)
        for j in range(1, N-1):
            P[i, j] = norm.cdf((s[j] - mu - rho*s[i] + step/2) / sigma) - \
                      norm.cdf((s[j] - mu - rho*s[i] - step/2) / sigma)
    P[np.floor((N-1)/2+1).astype(int):, :] = P[0:np.ceil((N-1)/2).astype(int), :][::-1, ::-1]

    ps = np.sum(P, axis=1)
    P = P / ps[:, np.newaxis] # transition matrix

    # compute invariant distribution of labor process (by iteration)
    inv = np.ones(N) / N # initial guess
    tol=1E-11; maxit=100^4
    for it in range(maxit):
        inv_new = inv @ P
        if np.max(np.abs(inv_new - inv)) < tol: break  
        inv = inv_new # invariant distribution

    return s, P, inv

In [186]:
def setPar(
    beta = 0.8,
    rho = 0.9,
    sigma = 0.2,
    phi_mean = 1.0,
    alpha = 2/3,
    c_e = 40.0,
    c_f = 20.0,
    D_bar = 100.0,
    mu_g  = 0.0,
    sigma_g = 0.2,  
    nPhi = 101,
    w = 1.0
    ):

	# === SHOCK DISCRETIZATION
    phi_mean = phi_mean*(1-rho)
    gPhi, F_trans, invPhi = tauchen(nPhi, rho, sigma, mu = phi_mean, m = 4.0)	
    gPhi = np.exp(gPhi)

	# === ENTRANTS DISTRIBUTION: Assume they draw from the invariant distribution.
    G_prob = invPhi

	# create dictionary with parameters
    param = {}
    param['alpha'] = alpha; param['beta'] = beta; param['F_trans'] = F_trans; param['gPhi'] = gPhi; param['nPhi'] = nPhi
    param['c_e'] = c_e; param['c_f'] = c_f; param['D_bar'] = D_bar; param['G_prob'] = G_prob; param['w'] = w

    return param

param = setPar()
print(param)


{'alpha': 0.6666666666666666, 'beta': 0.8, 'F_trans': array([[0.20443136, 0.05588705, 0.06285918, ..., 0.        , 0.        ,
        0.        ],
       [0.16082424, 0.0488525 , 0.0566335 , ..., 0.        , 0.        ,
        0.        ],
       [0.12378837, 0.04155719, 0.04965488, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.04965488, 0.04155719,
        0.12378837],
       [0.        , 0.        , 0.        , ..., 0.0566335 , 0.0488525 ,
        0.16082424],
       [0.        , 0.        , 0.        , ..., 0.06285918, 0.05588705,
        0.20443136]]), 'gPhi': array([ 0.43373312,  0.44994976,  0.46677271,  0.48422465,  0.5023291 ,
        0.52111044,  0.54059399,  0.560806  ,  0.5817737 ,  0.60352536,
        0.62609028,  0.64949887,  0.67378267,  0.69897441,  0.72510803,
        0.75221875,  0.78034309,  0.80951897,  0.83978568,  0.87118403,
        0.90375631,  0.93754642,  0.9725999 ,  1.00896397,  1.04668764,


In [123]:
def solve_bellman(p_guess, param):
    # unpacking parameters
    gPhi = param['gPhi']; F_trans = param['F_trans']; alpha = param['alpha']; nPhi = param['nPhi']
    c_f = param['c_f']; beta = param['beta']; w = param['w']
    
    # static decision:
    gN = (p_guess * alpha * gPhi / w)**(1 / (1.0 - alpha))
    gPi = p_guess * gPhi * gN**alpha - w * gN - c_f * w
    
    # solve bellman
    tol = 1e-9
    max_iter = 500
    iter_count = 10
    print_it = False  # set true to print iterations
    V = gPi  # initial guess
    v_guess = np.zeros(nPhi)

    for iter in range(0, max_iter):
        v_guess[:] = V    
        V = gPi + beta * np.maximum(0, F_trans.dot(v_guess)) 
        sup = np.max(np.abs(V - v_guess))  # check tolerance
        if sup < tol * np.max(np.abs(V)):
            if print_it: print(f"Iter: {iter}. Tol. achieved: {sup:.2E}")
            break
        if iter == max_iter and print_it: print(f"Max iterations achieved. VF did not converge: {sup:.2E}")
        if iter % iter_count == 0 and print_it: print(f"Iter: {iter}. Tol: {sup:.2E}")  
        
    chi = np.zeros(nPhi)
    chi[F_trans.dot(V) < 0] = 1.0  # recover exit policy function 

    return V, chi, gN, gPi


V, chi, gN, gPi = solve_bellman(2.0, param) # test
#print("V ", V)
#print("chi ", chi)
#print(gN)

In [136]:
def solve_price(param):
    def entry(p_guess): # 
        c_e = param['c_e']; beta = param['beta']; G_prob = param['G_prob']; w = param['w']
        V, chi, gN, gPi = solve_bellman(p_guess, param)
        excess_entry = beta * np.sum(V * G_prob) - c_e * w
        return excess_entry
    
    p0 = 0.05; p1 = 10.0 # guess: lower and upper bound. Might have to change for diff. parameters
    p = brentq(entry, 0.05, 10.0)

    V, chi, gN, gPi = solve_bellman(p, param)

    return p, V, chi, gN, gPi

sol =solve_price(param)
#print(sol[0]) # price

1.486168320887955


In [173]:
# unpacking parameters
gPhi = param['gPhi']; F_trans = param['F_trans']; alpha = param['alpha']; nPhi = param['nPhi']
G_prob = param['G_prob']; D_bar = param['D_bar']
p = sol[0]; chi = sol[2]; gN = sol[3];

# construct transition probability:
Phat = ((1 - chi) * F_trans.T)

matG = np.array([G_prob]).T

M=1
#np.linalg.inv(np.eye(nPhi) - Phat) * G_prob
#test = M*G_prob @ np.linalg.inv(np.eye(nPhi) - Phat).T
def inv_dist(M):
    return M*G_prob @ np.linalg.inv(np.eye(nPhi) - Phat).T

test =  inv_dist(M)


#Phat = ((1 - chi)*F_trans)
#print(F_trans)
#Phat
#Phat = ((1 .- chi).*F_trans)'

# supply: integrate the total production
y = gPhi * gN**alpha
supply = np.sum(inv_dist(1) * y)  # just use the function with an arbitrary M
supply

782.3460629901617

In [175]:
def solve_m(param, solution):
    # unpacking parameters
    gPhi = param['gPhi']; F_trans = param['F_trans']; alpha = param['alpha']; nPhi = param['nPhi']
    G_prob = param['G_prob']; D_bar = param['D_bar']
    p = solution[0]; chi = solution[2]; gN = solution[3];
    
    # construct transition probability:
    Phat = ((1 - chi) * F_trans.T)  # note the transpose here, only firms with z>z* keep operating
    
    # invariant distribution is just a homogeneous function of M
    def inv_dist(M):
        return M*G_prob @ np.linalg.inv(np.eye(nPhi) - Phat).T
    
    # supply: integrate the total production
    y = gPhi * gN**alpha
    supply = np.sum(inv_dist(1) * y)  # just use the function with an arbitrary M
    
    # demand
    demand = D_bar / p
    
    # find mass of entrants (exploit linearity of the invariant distribution)
    M = demand / supply
    mu = inv_dist(M)
    
    return M, mu


M, mu = solve_m(param, sol)
#print(M)
#print(mu)
#np.sum(mu)

0.08600686129049144
[2.69224126e-06 1.22264324e-06 1.69695499e-06 2.32777238e-06
 3.15971729e-06 4.24871467e-06 5.66437624e-06 7.49263432e-06
 9.83857846e-06 1.28294379e-05 1.66176423e-05 2.13838786e-05
 2.73400381e-05 3.47319187e-05 4.38415070e-05 5.49886303e-05
 6.85317289e-05 8.48674774e-05 1.04428971e-04 1.27682206e-04
 1.55120639e-04 1.87257688e-04 2.24617247e-04 2.67722577e-04
 3.17084537e-04 3.73191017e-04 4.36500975e-04 5.07448818e-04
 5.86468144e-04 6.74048210e-04 7.70841389e-04 8.77844246e-04
 9.96676688e-04 1.12997979e-03 1.28193964e-03 1.45891879e-03
 1.67013684e-03 1.92829154e-03 2.24995923e-03 2.65557473e-03
 3.16878537e-03 3.81501923e-03 4.61921424e-03 5.60281559e-03
 6.78033736e-03 8.15595695e-03 9.72071432e-03 1.14508812e-02
 1.33079272e-02 1.52402602e-02 1.71866042e-02 1.90805840e-02
 2.08558700e-02 2.24511646e-02 2.38143883e-02 2.49056172e-02
 2.56985832e-02 2.61808001e-02 2.63525724e-02 2.62252440e-02
 2.58190537e-02 2.51609088e-02 2.42822900e-02 2.32174077e-02
 2.2

0.6412681312285025

In [180]:
def ModelStats(param, sol_price, M, mu, Printa = True):
    # unpacking parameters
    gPhi = param['gPhi']; F_trans = param['F_trans']; alpha = param['alpha']; nPhi = param['nPhi']
    G_prob = param['G_prob']; D_bar = param['D_bar']
    p = sol_price[0]; chi = sol_price[2]; gN = sol_price[3]; gPi = sol_price[4]

    # productivity distribution
    pdf_dist = mu / np.sum(mu)
    cdf_dist = np.cumsum(pdf_dist)
    
    # employment distribution
    emp_dist = mu * gN
    pdf_emp = emp_dist / np.sum(emp_dist)
    cdf_emp = np.cumsum(pdf_emp)
    
    # exit productivity
    cut_index = np.flatnonzero(chi == 0)[0]
    phicut = gPhi[cut_index]

    # stats
    avg_firm_size = np.sum(emp_dist) / np.sum(mu)
    exit_rate = M / np.sum(mu)
    Y = np.sum((gPhi * gN**alpha) * mu)  # agg production
    emp_prod = np.sum(emp_dist)  # employment used in production
    Pi = np.sum(gPi * mu)  # aggregate profits

    if Printa==True:
        print("Model Stats")
        print("Price: ", p)
        print("Avg. Firm Size: ", avg_firm_size)
        print("Exit/entry Rate: ", exit_rate)
        print("Productivity Cutoff: ", phicut)
        print("Aggregate Output: ", Y)
        print("Aggregate Profits: ", Pi)

    
    return (pdf_dist, cdf_dist, pdf_emp, cdf_emp, 
            avg_firm_size, exit_rate, Y, emp_prod, phicut, Pi)

Stats = ModelStats(param, sol, M, mu)

Model Stats
Price:  1.486168320887955
Avg. Firm Size:  103.9606732661901
Exit/entry Rate:  0.13411996807906973
Productivity Cutoff:  2.620312230399254
Aggregate Output:  67.28712932075692
Aggregate Profits:  20.507970708763292


In [189]:
### PS. The function is not considering the corner solution where there is no entry. Change the function later.

def SolveModel(param):
    # Solve For Prices
	sol_price =solve_price(param)  
	M, mu = solve_m(param, sol_price)

	if M<=0: print("Warning: No entry, eq. not found.")
	stats = ModelStats(param, sol_price, M, mu)
	return (sol_price, M, mu, stats)

results = SolveModel(param)

Model Stats
Price:  1.486168320887955
Avg. Firm Size:  103.9606732661901
Exit/entry Rate:  0.13411996807906973
Productivity Cutoff:  2.620312230399254
Aggregate Output:  67.28712932075692
Aggregate Profits:  20.507970708763292


In [190]:
# increase entry cost: 
results2 = SolveModel(setPar(c_e = 60))


Model Stats
Price:  1.5973485530259657
Avg. Firm Size:  120.56389584648885
Exit/entry Rate:  0.1061393447863616
Productivity Cutoff:  2.4348385434435036
Aggregate Output:  62.60374406735665
Aggregate Profits:  22.274190594357524


In [191]:
# increase fixed cost: 
results2 = SolveModel(setPar(c_f = 30))


Model Stats
Price:  1.597370311025299
Avg. Firm Size:  142.4103738500016
Exit/entry Rate:  0.18950685121843872
Productivity Cutoff:  2.92534679145905
Aggregate Output:  62.60289133320208
Aggregate Profits:  19.28941261371943
